# 06 - HyDe (Hypothetical Document Embeddings)

**Complexity:** ⭐⭐⭐

**Use Cases:** Ambiguous queries, domain jargon, queries with abbreviations

**Key Feature:** Generates hypothetical "perfect answer" document, embeds it, uses for retrieval.

**Example:**
```
Query: "How does MMR work?"

Hypothetical Doc:
"MMR (Maximal Marginal Relevance) balances relevance with diversity by
iteratively selecting documents that are relevant to query AND dissimilar
to already selected documents..."

→ Embedding this detailed description finds better semantic matches
```

In [ ]:
import sys
sys.path.append('../..')

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from shared.config import OPENAI_VECTOR_STORE_PATH, DEFAULT_MODEL
from shared.utils import load_vector_store, print_section_header, format_docs
from shared.prompts import HYDE_PROMPT, RAG_PROMPT_TEMPLATE
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

print_section_header("Setup: HyDe")

embeddings = OpenAIEmbeddings()
vectorstore = load_vector_store(OPENAI_VECTOR_STORE_PATH, embeddings)
llm = ChatOpenAI(model=DEFAULT_MODEL, temperature=0)

print("✅ Setup complete!")

## 2. HyDe Document Generator

In [ ]:
print_section_header("HyDe Generator")

# Create HyDe document generator
hyde_generator = HYDE_PROMPT | llm | StrOutputParser()

# Test
query = "What is semantic search?"
print(f"Query: '{query}'\n")

hypo_doc = hyde_generator.invoke({"question": query})
print("Generated Hypothetical Document:")
print("=" * 80)
print(hypo_doc)
print("=" * 80)

## 3. HyDe Retrieval

In [ ]:
from shared.utils import print_results

print_section_header("HyDe vs Standard Retrieval")

query = "How to improve retrieval quality?"

# Standard retrieval
print("[STANDARD RETRIEVAL]")
standard_docs = vectorstore.similarity_search(query, k=3)
print_results(standard_docs, max_docs=2, preview_length=120)

# HyDe retrieval
print("\n" + "=" * 80)
print("\n[HYDE RETRIEVAL]")
hypo_doc = hyde_generator.invoke({"question": query})
print(f"\nGenerated doc preview: {hypo_doc[:200]}...\n")
hyde_docs = vectorstore.similarity_search(hypo_doc, k=3)
print_results(hyde_docs, max_docs=2, preview_length=120)

print("\n💡 HyDe often finds more semantically relevant documents")

## 4. HyDe RAG Chain

In [ ]:
print_section_header("HyDe RAG Chain")

def hyde_retrieve(query: str):
    hypo_doc = hyde_generator.invoke({"question": query})
    docs = vectorstore.similarity_search(hypo_doc, k=4)
    return docs

hyde_retriever = RunnableLambda(hyde_retrieve)

hyde_chain = (
    {"context": hyde_retriever | format_docs, "input": RunnablePassthrough()}
    | RAG_PROMPT_TEMPLATE
    | llm
    | StrOutputParser()
)

print("✓ HyDe RAG chain created")

# Test
query = "Best practices for chunk sizing?"
print(f"\nQuery: '{query}'\n")
print("=" * 80)

response = hyde_chain.invoke(query)
print(response)
print("\n" + "=" * 80)

## Summary

**Flow:**
```
Query → Generate Hypo Doc → Embed → Retrieve → LLM → Response
```

**Advantages:**
✅ Better for ambiguous queries  
✅ Handles jargon and abbreviations  
✅ Improves semantic matching  
✅ Works with specialized domains  

**Limitations:**
- Extra LLM call (cost + latency)
- May hallucinate in hypo doc
- Not always better than standard

**When to Use:**
- Vague or ambiguous queries
- Technical jargon
- Queries with abbreviations

**Next:** [07_adaptive_rag.ipynb](07_adaptive_rag.ipynb) - Intelligent query routing